In [ ]:
# TPU
# !pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.10-cp37-cp37m-linux_x86_64.whl

In [9]:
!pip install transformers sentence_transformers

In [10]:
from transformers import AutoTokenizer, AutoModel
import torch
import pickle
from sentence_transformers import util
from datetime import datetime

In [3]:
from google.colab import drive
drive.mount('/content/drive')
#drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Load pretrained 

- multilingual sentence transformers from checkpoint
- tokenizer from checkpoint

In [3]:
multilingual_checkpoint = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
tokenizer = AutoTokenizer.from_pretrained(multilingual_checkpoint)
model = AutoModel.from_pretrained(multilingual_checkpoint)

In [ ]:
# GPU
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
print(device)

In [ ]:
# TPU
# unfortunately incompatible wheel package for pytorch-xla 1.10 version
#import torch_xla.core.xla_model as xm
#device = xm.xla_device()
#print(device)
#pip list | grep torch

In [23]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

def calculateEmbeddings(sentences,tokenizer,model,device="cpu"):
    tokenized_sentences = tokenizer(sentences, padding=True, truncation=True, max_length=128, return_tensors='pt')
    tokenized_sentences.to(device)
    with torch.no_grad():
        model_output = model(**tokenized_sentences)
    sentence_embeddings = mean_pooling(model_output, tokenized_sentences['attention_mask'])
    del tokenized_sentences
    torch.cuda.empty_cache()
    return sentence_embeddings

def findTopKMostSimilar(query_embedding, embeddings, k):
    cosine_scores = util.pytorch_cos_sim(query_embedding, embeddings)
    cosine_scores_list = cosine_scores.squeeze().tolist()
    pairs = []
    for idx,score in enumerate(cosine_scores_list):
        pairs.append({'index': idx, 'score': score})
    pairs = sorted(pairs, key=lambda x: x['score'], reverse=True)
    return pairs[0:k]

def saveToDisc(embeddings, output_filename):
    with open(output_filename, "ab") as f:
        pickle.dump(embeddings, f, protocol=pickle.HIGHEST_PROTOCOL)

# Create sentence embeddings


*   Load sentences from raw text file
*   Precalculate in batches of 1000, to avoid running out of memory
*   Save to disc/files incrementally, to be able to reuse later (in total 5 files of 100.000 embedding each)



In [ ]:
batch_size = 1000

raw_text_file = '/content/drive/MyDrive/huggingface/shortened_abstracts_hu_2021_09_01.txt'
datetime_formatted = datetime.now().strftime('%Y-%m-%d_%H:%M:%S')
output_embeddings_file_batched = f'/content/drive/MyDrive/huggingface/embeddings_{batch_size}_batches_at_{datetime_formatted}.pkl'
output_embeddings_file = f'/content/drive/MyDrive/huggingface/embeddings_at_{datetime_formatted}.pkl'

print(datetime.now())
concated_sentence_embeddings = None
all_sentences = []
line = 'init'
total_read = 0
total_read_limit = 500000
skip_index = 400000
with open(raw_text_file) as f:
    while line and total_read < total_read_limit:
        count = 0
        sentence_batch = []
        while line and count < batch_size:
            line = f.readline()
            sentence_batch.append(line)
            count += 1
        
        all_sentences.extend(sentence_batch)
        
        if total_read >= skip_index:
          sentence_embeddings = calculateEmbeddings(sentence_batch,tokenizer,model,device)
          if concated_sentence_embeddings == None:
              concated_sentence_embeddings = sentence_embeddings
          else:
              concated_sentence_embeddings = torch.cat([concated_sentence_embeddings, sentence_embeddings], dim=0)
          print(concated_sentence_embeddings.size())
          saveToDisc(sentence_embeddings,output_embeddings_file_batched)
        total_read += count
print(datetime.now())

# Test: Query embeddings

In [ ]:
query_embedding = calculateEmbeddings(['Melyik a legnépesebb város a világon?'],tokenizer,model,device)
top_pairs = findTopKMostSimilar(query_embedding, concated_sentence_embeddings, 5)

for pair in top_pairs:
    i = pair['index']
    score = pair['score']
    print("{} \t\t Score: {:.4f}".format(all_sentences[skip_index+i], score))

# Test: Load pre-calculated embeddings

*   Load embedding from files and stitch them together
*   Save into one file


In [20]:
def concatTensors(new_tensor, acc_tensor='None'):
  if acc_tensor == None:
    acc_tensor = new_tensor
  else:
    acc_tensor = torch.cat([acc_tensor, new_tensor], dim=0)
  return acc_tensor

def loadFromDisc(batch_size, number_of_batches, filename):
    concated_sentence_embeddings = None
    count = 0
    batches = 0
    with open(filename, "rb") as f:
        loaded_embeddings = torch.empty([batch_size])
        while count < number_of_batches and loaded_embeddings.size()[0]==batch_size:
            loaded_embeddings = pickle.load(f)
            count += 1
            concated_sentence_embeddings = concatTensors(loaded_embeddings,concated_sentence_embeddings)
    print(f'Read file using {count} number of read+unpickle operations')
    print(concated_sentence_embeddings.size())
    return concated_sentence_embeddings


output_embeddings_file = 'data/processed/DBpedia_shortened_abstracts_hu_embeddings.pkl'

embeddings_files = [
  '/content/drive/MyDrive/huggingface/embeddings_1000_batches_at_2021-11-23_08:17:17.pkl',
  '/content/drive/MyDrive/huggingface/embeddings_1000_batches_at_2021-11-23_08:28:46.pkl',
  '/content/drive/MyDrive/huggingface/embeddings_1000_batches_at_2021-11-23_08:40:54.pkl',
  '/content/drive/MyDrive/huggingface/embeddings_1000_batches_at_2021-11-23_08:56:26.pkl',
  '/content/drive/MyDrive/huggingface/embeddings_1000_batches_at_2021-11-23_09:31:47.pkl'
]

all_embeddings = None
for idx,emb_file in enumerate(embeddings_files):
  print(f'Processing file {idx}')
  file_embeddings = loadFromDisc(1000, 100, emb_file)
  all_embeddings = concatTensors(file_embeddings,all_embeddings)

print(all_embeddings.size())

Processing file 0
Read file using 100 number of read+unpickle operations
torch.Size([100000, 384])
Processing file 1
Read file using 100 number of read+unpickle operations
torch.Size([100000, 384])
Processing file 2
Read file using 100 number of read+unpickle operations
torch.Size([100000, 384])
Processing file 3
Read file using 100 number of read+unpickle operations
torch.Size([100000, 384])
Processing file 4
Read file using 67 number of read+unpickle operations
torch.Size([66529, 384])
torch.Size([466529, 384])


In [28]:
all_embeddings_output_file = '/content/drive/MyDrive/huggingface/shortened_abstracts_hu_2021_09_01_embedded.pt'
#saveToDisc(all_embeddings, all_embeddings_output_file)
torch.save(all_embeddings,all_embeddings_output_file)